# Constrained Search Space

> Generate constrained search space. Alert if constraints are not satisfiable.

In [ ]:
#| default_exp utils.search_space_helper

In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
#| export
import numpy as np
from typing import Generator, List, Tuple, Protocol
from dataclasses import dataclass
from collections import OrderedDict

In [ ]:
#| export
class Trial(Protocol):
    "Protocol for a trial object"
    @staticmethod
    def suggest_float(name: str, *args, **kwargs) -> float:
        ...

In [ ]:
#| export
@dataclass
class ConstrainedSearchSpace:
    """
    A class that generates a search space with constraints
    """
    bounds: dict[str, tuple[float, float]] # bounds of each channel
    constraint: tuple[float, float] # constraint of the sum of all channels

    def __post_init__(self):
        self.bounds = OrderedDict(sorted(self.bounds.items(), key=lambda x: x[1]))
    
    def __call__(
        self, 
        trial: Trial # trial object
        ) -> dict[str, float]: # selected budget
        "Sample from constrained search space"
        selected_budget = {}
        bounds_values = list(self.bounds.values())
        bounds_items = list(self.bounds.items())
        for n, (name, bound) in enumerate(bounds_items[:-1]):
            curr_total = sum(selected_budget.values())
            new_min_bound = self.constraint[0]-(curr_total+sum(b[1] for b in bounds_values[n+1:]))
            new_max_bound = self.constraint[1]-(curr_total+sum(b[0] for b in bounds_values[n+1:]))
            updated_bounds = (
                    max(bound[0], new_min_bound), 
                    min(bound[1], new_max_bound)
            )
            selection = trial.suggest_float(name, *updated_bounds)
            selected_budget[name] = selection
        last = bounds_values[-1]
        last_name = bounds_items[-1][0]
        choice = trial.suggest_float(
            last_name, 
            max(last[0], self.constraint[0]-sum(selected_budget.values())),
            min(last[1], self.constraint[1]-sum(selected_budget.values()))
        )
        selected_budget[last_name] = choice
        return selected_budget

In [ ]:
show_doc(ConstrainedSearchSpace.__call__)

---

### ConstrainedSearchSpace.__call__

>      ConstrainedSearchSpace.__call__ (trial:__main__.Trial)

*Call self as a function.*

In [ ]:
class TestTrial:
    @staticmethod
    def suggest_float(name: str, low: float, high: float, **kwargs) -> float:
        return np.random.uniform(low, high)

In [ ]:
RNG = np.random.default_rng(44)
actual_spends = np.exp(RNG.normal(7, 2, 5)) # generate some random spends
bounds = {f"dim_{dim}": (np.round(.8*spend, 2), np.round(1.2*spend, 2)) for dim, spend in enumerate(actual_spends)} # generate bounds for each spend
total_spend = sum(actual_spends) # calculate the total spend
constraint = (total_spend, total_spend) # set the constraint to be the total spend
trial = TestTrial() # creates a mock trial
search_space = ConstrainedSearchSpace(bounds=bounds, constraint=constraint) # create the search space
selected_budget = search_space(trial) # get the first sample
for name, sample in selected_budget.items(): # iterate over the sample
    print(f"Sample {name}: {sample:.2f}, Bound {name}: {bounds[name]}") # print the sample and the bound
print(f"Sample Total: {sum(selected_budget.values()):.2f}, Total: {total_spend:.2f}") # print the total of the sample and the total spend

Sample dim_1: 1297.19, Bound dim_1: (1076.5, 1614.74)
Sample dim_2: 2059.89, Bound dim_2: (1686.42, 2529.63)
Sample dim_4: 5580.39, Bound dim_4: (4561.22, 6841.82)
Sample dim_3: 12345.41, Bound dim_3: (8516.67, 12775.01)
Sample dim_0: 18288.21, Bound dim_0: (15816.07, 23724.11)
Sample Total: 39571.09, Total: 39571.09


In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()